<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part4_custom_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom Search Agent**

In [29]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [30]:
%%capture --no-stderr
%pip install langchain_community ChatGoogleGenerativeAI langchain

In [22]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [23]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key, temperature=0)

[hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)



In [15]:

from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (create_react_agent, AgentExecutor)
from langchain import hub

from dotenv import load_dotenv
load_dotenv()

def lookup(name: str):
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0
    )
    template = """give the full name {name_of_person} I want you to get it me a link to their Linkedin profile page.
                  Your answer should contain only a URL"""

    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

    tools_for_agent = [
        Tool(
            name="Crawl Google 4 linkedin profile page",
            func="?",
            description="useful for when you need get the Linkedin Page URL"
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(
        llm=llm,
        tools=tools_for_agent,
        prompt=react_prompt
    )

In [16]:
linkedin_url = lookup(name="Eden Marco")
print(linkedin_url)

ValidationError: 1 validation error for Tool
func
  Input should be callable [type=callable_type, input_value='?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/callable_type

In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (create_react_agent, AgentExecutor)
from langchain import hub

from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv

load_dotenv()

def lookup(name: str):
    # Define the LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0
    )

    # Define the prompt template
    template = """Give the full name {name_of_person} and provide a link to their LinkedIn profile page.
                  Your answer should contain only a URL."""
    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

    # Define a function for the tool
    def google_search(query: str):
        # Implement logic to query Google for LinkedIn profiles (stubbed for now)
        # Example: use an API like SerpAPI or a custom scraper
        return f"https://www.linkedin.com/in/{query.replace(' ', '').lower()}"

    # Create the tool
    tools_for_agent = [
        Tool(
            name="Crawl Google for LinkedIn profile page",
            func=google_search,
            description="Useful for retrieving a LinkedIn profile URL."
        )
    ]

    # Pull the prompt template from LangChain Hub
    react_prompt = hub.pull("hwchase17/react")

    # Create the agent
    agent = create_react_agent(
        llm=llm,
        tools=tools_for_agent,
        prompt=react_prompt
    )

    # Execute the agent with the input name
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent)
    result = agent_executor.run(name_of_person=name)
    return result



In [ ]:
# Call the lookup function
linkedin_url = lookup(name="Eden Marco")
print(linkedin_url)
